# Maximum Likelihood Estimation

This notebook illustrates maximum likelihood estimation and how to calculate different standard errors (from the information matrix, the gradients and the "sandwich" approach).

The application is very simple: estimating the mean and variance of a random variable. Some of the subsequent chapters (notebooks) work with more complicated models, for instance, GARCH models.

## Load Packages and Extra Functions

The notebook first implements MLE step-by-step. At the end it also presents the `MLE()` function from the (local) `FinEcmt_MLEGMM` module that wraps those calculations.

For the numerical optimization we use the [Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl) package and for calculating derivatives the [FiniteDiff.jl](https://github.com/JuliaDiff/FiniteDiff.jl) package. ([ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl) is an alternative.)

In [1]:
MyModulePath = joinpath(pwd(),"src")
!in(MyModulePath,LOAD_PATH) && push!(LOAD_PATH,MyModulePath)
using FinEcmt_OLS
using FinEcmt_MLEGMM: MLE     #load only the MLE() function

In [2]:
#=
include(joinpath(pwd(),"src","FinEcmt_OLS.jl"))
include(joinpath(pwd(),"src","FinEcmt_MLEGMM.jl"))
using .FinEcmt_OLS
using .FinEcmt_MLEGMM: MLE     #load only the MLE() function
=#

In [3]:
using LinearAlgebra, DelimitedFiles, Statistics, Optim

#loading and renaming some functions from FiniteDiff (to get shorter names)
using FiniteDiff: finite_difference_hessian as hessian, finite_difference_jacobian as jacobian

## Loading Data

In [4]:
xx  = readdlm("Data/FFdSizePs.csv",',',skipstart=1)
x   = xx[:,2]                 #returns for the portfolio of the smallest firms
xx  = nothing

## Traditional Estimates

of the mean $\mu$ and the variance $\sigma^2$.

The standard errors of the mean and standard deviation are from traditional textbook formulas.

In [5]:
T = length(x)

(μ_trad,σ²_trad) = (mean(x),var(x,corrected=false))    #corrected=false gives 1/T formula, not 1/(T-1)

std_trad = sqrt.([σ²_trad,2*σ²_trad^2]/T)   #standard errors, textbook formulas

printblue("Traditional estimates and their std:\n")
xx = [[μ_trad,σ²_trad] std_trad]
printmat(xx;colNames=["estimate","std"],rowNames=["μ","σ²"])

Traditional estimates and their std:

    estimate       std
μ      0.042     0.010
σ²     0.840     0.013



# Point Estimates from ML

The next few cells define a log likelihood function and estimate the coefficients by maximizing it.

## The (log) Likelihood Function for Estimating the Parameters of a N(,)

### A Remark on the Code

- `(μ,σ²) = par` splits up the vector `par` into two numbers (for the mean and variance)

In [6]:
"""
    NormalLL(par,x)

Calculate log likelihood for a `N(μ,σ²)` distribution.

`par = [μ,σ²]` is a vector with the parameters , `x` is a vector with data
"""
function NormalLL(par,x)
    (μ,σ²) = par
    σ      = sqrt(σ²)
    z      = (x .- μ)./σ
    LLt    = logpdfNorm.(z) .- log(σ)            #shorter, from local module
    #LLt   = -(1/2)*log(2*pi) - (1/2)*log(σ²) .- (1/2)*abs2.(x.-μ)/σ²  #vector, all x[t]
    return LLt
end;

## Try the Likelihood Function

In [7]:
par0 = [0.0,1.0]           #initial parameter guess of [μ,σ²]

LLt = NormalLL(par0,x)     #just trying the log likelihood fn

printlnPs("log likelihood value at par0: ",sum(LLt))

log likelihood value at par0: -11155.385


## Optimize the Likelihood Function

In [8]:
Sol = optimize(par->-sum(NormalLL(par,x)),par0)  #minimize -sum(LLt)
parHat = Optim.minimizer(Sol)                    #the optimal solution

printlnPs("log-likelihood at point estimate (compare with the value above): ",-Optim.minimum(Sol))

printblue("\nParameter estimates:\n")
xx = [[μ_trad,σ²_trad] parHat]
printmat(xx;colNames=["traditional","MLE"],rowNames=["μ","σ²"],width=13)

log-likelihood at point estimate (compare with the value above): -11088.409

Parameter estimates:

    traditional          MLE
μ         0.042        0.042
σ²        0.840        0.840



# Standard Errors I: Information Matrix 

If the likelihood function is correctly specified, then MLE is typically asymptotically normally distributed as

$\sqrt{T}(\hat{\theta}-\theta)  \rightarrow^{d}N(0,V) \: \text{, where } \: V=I(\theta)^{-1}\text{ with }$

$I(\theta) =-\text{E}\frac{\partial^{2}\ln L_t}{\partial\theta\partial\theta^{\prime}}$

where $I(\theta)$ is the information matrix (*not* an identity matrix) and $\ln L_t$  is the contribution of period $t$ to the log likelihood function.

### A Remark on the Code

The code below calculates numerical derivatives. It does so by noticing that
$\text{E}\frac{\partial^{2}\ln L_t}{\partial\theta\partial\theta^{\prime}} = 
\frac{\partial^{2}\text{E}\ln L_t}{\partial\theta\partial\theta^{\prime}},$
so we can differentiate the mean (across data points) log likelihood value. (Clearly, we could equally well calculate $T$ different $2\times 2$ matrices and then average.)

- `hessian(par->mean(NormalLL(par,x)),parHat)` calculates the 2nd derivatives of the average log-likelihood function.

In [9]:
Ia = -hessian(par->mean(NormalLL(par,x)),parHat)  #(-1) * 2nd derivatives of mean(LLt)

Ia       = (Ia+Ia')/2         #to guarantee symmetry, fixes possible rounding errors
vcv      = inv(Ia)/T          #variance-covariance matrix of estimates (not sqrt(T)*estimates)
std_hess = sqrt.(diag(vcv))

printblue("standard errors:\n")
xx = [std_trad std_hess]
printmat(xx;colNames=["traditional","MLE (InfoMat)"],rowNames=["μ","σ²"],width=18)

standard errors:

         traditional     MLE (InfoMat)
μ              0.010             0.010
σ²             0.013             0.013



# Standard Errors II: Gradients and Sandwich

An alternative way of calculating the information matrix is to use the outer product of the gradients:

$J(\theta)=\text{E}\left[  \frac{\partial\ln L_t}{\partial\theta
}\frac{\partial\ln L_t}{\partial\theta^{\prime}}\right]$

This would coincide with $I(\theta)$ as defined above, in a very large sample and if the model is correctly specified.

### A Remark on the Code

The code below fills row $t$ of a $T\times 2$ matrix (called `δL`) with 
$\frac{\partial\ln L_t}{\partial\theta}.$
For each $t$, the outer product is a $2\times2$ matrix, and then we average across $t$. This is done by calculating 
`J = δL'δL/T`. A loop would also work well.

## Std from the Gradients

In [10]:
δL = jacobian(par->NormalLL(par,x),parHat)   #Tx2
J  = δL'δL/T               #2xT * Tx2

vcv       = inv(J)/T
std_grad  = sqrt.(diag(vcv))                          #std from gradients

printblue("standard errors:\n")
xx = [std_trad std_hess std_grad]
printmat(xx;colNames=["traditional","MLE (InfoMat)","MLE (gradients)"],rowNames=["μ","σ²"],width=18)

standard errors:

         traditional     MLE (InfoMat)   MLE (gradients)
μ              0.010             0.010             0.010
σ²             0.013             0.013             0.005



## Std from the Sandwich Estimator

We could also use the "sandwich" estimator

$V=I(\theta)^{-1}J(\theta)I(\theta)^{-1}.$

When the model (likelihood function) is misspecified, then the three variance-covariance matrices may differ, and the sandwich approach is often the most robust.

In [11]:
vcv       = inv(Ia) * J * inv(Ia)/T
std_sandw = sqrt.(diag(vcv))                          #std from sandwich

printblue("standard errors:\n")
xx = [std_trad std_hess std_grad std_sandw]
printmat(xx,colNames=["traditional","MLE (InfoMat)","MLE (gradients)","MLE (sandwich)"],rowNames=["μ","σ²"],width=18)

standard errors:

         traditional     MLE (InfoMat)   MLE (gradients)    MLE (sandwich)
μ              0.010             0.010             0.010             0.010
σ²             0.013             0.013             0.005             0.036



Try this: replace the data series `x` with simulated data from a $N()$ distribution. Then, do the different standard errors get closer to each other?

# A Function for MLE (extra)

The next cell uses a function which combines the computations of several of the cells above. 

It requires a function for the log-likehood function as written above, that is, taking `(par,x)` as inputs and generating a T-vector `LLt` as output.

In [12]:
@doc2 MLE

```
MLE(LLtFun,par0,y,x,lower,upper)
```

Calculate ML point estimates of K parameters and three different types of standard errors: from the Information matrix, from the gradients and the sandwich approach.

### Input

  * `LLtFun::Function`:  name of log-likelihood function
  * `par0::Vector`:      K-vector, starting guess of the parameters
  * `y::VecOrMat`:       vector or matrix with the dependent variable
  * `x::VecOrMat`:       vector or matrix with data, use `nothing` if not needed
  * `lower::Vector`:     lower bounds on the parameters, nothing or fill(-Inf,K) if no bounds
  * `upper::Vector`:     upper bounds on the parameters, nothing or fill(Inf,K) if no bounds

### Requires

  * `using FiniteDiff: finite_difference_hessian as hessian, finite_difference_jacobian as jacobian`

### Notice

The `LLtFun` should take `(par,y,x)` as inputs and generate a T-vector `LLt` as output.


In [13]:
#using CodeTracking
#println(@code_string MLE(NormalLL,[1],[1],[1]))    #print the source code

In [14]:
LLtFun(par,y,x) = NormalLL(par,y)
(parHat,std_hess,std_grad,std_sandw,LL_t) = MLE(LLtFun,par0,x,nothing)

printblue("point estimates and standard errors:\n")
xx = [parHat std_hess std_grad std_sandw]
printmat(xx;colNames=["estimate","std (InfoMat)","std (gradients)","std (sandwich)"],rowNames=["μ","σ²"],width=18)

point estimates and standard errors:

            estimate     std (InfoMat)   std (gradients)    std (sandwich)
μ              0.042             0.010             0.010             0.010
σ²             0.840             0.013             0.005             0.036

